In [ ]:
from sepal_ui import sepalwidgets as sw

import ipyvuetify as v
from utils import messages as ms

In [ ]:
#input and output as mutable object 
class Bands_input_io:
    def __init__(self):        
        #output 
        self.forest_bands = None
        self.non_forest_bands = None
        self.bin_map = False

bi_io = Bands_input_io()

In [ ]:
#create the output 
bi_output = sw.Alert().add_msg(ms.BANDS_INTRO)

#create the button 
bi_btn = sw.Btn(ms.BANDS_BTN)

#create the inputs 
bi_select_f = v.Select(items=None, multiple=True, label='foreground classes', v_model=None)
bi_select_nf = v.Select(multiple=True, items=None, label='background classes', v_model=None)

#bind the widgets
bi_output.bind(bi_select_f, bi_io, 'forest_bands')
bi_output.bind(bi_select_nf, bi_io, 'non_forest_bands')

#create a bands selector tile
id_ = "input_widget"
title = "Select binary map classes"

bi_tile = sw.Tile(
    id_,
    title,
    btn=bi_btn,
    inputs=[bi_select_f,bi_select_nf],
    output=bi_output
)

In [ ]:
#function to reset the bands 
def reset_bands(values, f = bi_select_f, nf = bi_select_nf):
    
    #reset the output
    setattr(bi_io, 'forest_bands', None)
    setattr(bi_io, 'forest', None)
    
    #reset the select 
    f.v_model = None
    nf.v_model = None
    
    #add the new values 
    f.items = values
    nf.items = values
    
    return

In [ ]:
#validate the band selection 
from scripts import run
from functools import partial

def process_start(widget, event, data, output):
    
    #toggle the loading button 
    widget.toggle_loading()
    
    #load inputs 
    file = getattr(fi_io, 'file')
    values = getattr(fi_io, 'values')
    forest = getattr(bi_io, 'forest_bands')
    nforest = getattr(bi_io, 'non_forest_bands')
    
    #check input
    if not output.check_input(file, ms.NO_FILE): return widget.toggle_loading()
    if not output.check_input(forest, ms.NO_BANDS): return widget.toggle_loading()
    if not output.check_input(nforest, ms.NO_BANDS): return widget.toggle_loading()
    
    #check if overwrite 
    for val in forest:
        if val in nforest:
            output.add_msg(ms.BANDS_OVERRIDE, 'error')
            widget.toggle_loading()
            return 
        
    #create the bin map 
    #try:
    bin_map = run.set_bin_map(
        raster  = file,
        values  = values,
        forest  = forest,
        nforest = nforest,
        output  = output
    )    
    
    #set the output 
    setattr(bi_io, 'bin_map', bin_map)
        
    #except Exception as e: 
    #    output.add_live_msg(str(e), 'error')
        
    #toggle the loading button 
    widget.toggle_loading()
    
    return 

bi_btn.on_event('click', partial(
    process_start,
    output= bi_output
))
    

In [ ]:
bi_tile